In [6]:
import os
import firebirdsql

from openpyxl import Workbook
from openpyxl.styles import Alignment
from collections import defaultdict
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

True

In [5]:
def get_firebird_connection():
    # Ajustar com os parâmetros corretos do Firebird, inclusive charset
    return firebirdsql.connect(
        host=os.getenv('HOST'),
        port=int(os.getenv('PORT', '3050')),
        database=os.getenv('DB_PATH'),
        user=os.getenv('APP_USER'),
        password=os.getenv('PASSWORD'),
        role=os.getenv('ROLE'),
        auth_plugin_name=os.getenv('AUTH'),
        wire_crypt=False,
        charset='ISO8859_1'
    )

In [13]:
def gerar_planilha_similaridade(output_path="similaridades.xlsx"):
    """
    Gera uma planilha Excel relacionando os produtos (PEÇAS) e suas similaridades.
    Cria uma célula mesclada para o nome da peça na coluna A e coloca as similaridades
    na coluna B, com uma linha de separação após cada grupo.
    """

    # 1. Conexão com o Firebird
    conn = get_firebird_connection()
    cursor = conn.cursor()

    # 2. Query para buscar os produtos e suas similaridades.
    #
    #    - Ajuste as colunas conforme sua estrutura real:
    #      por exemplo, se quiser o nome do produto, verifique como se chama a coluna da tabela PRODUTO.
    #    - Se quiser apenas o CDPRODUTO, mude no SELECT para p.CDPRODUTO em vez de p.NMPRODUTO
    #      ou qualquer outro identificador.

    query = """
        SELECT 
            p.NUMORIGINAL,  -- ou p.CDPRODUTO se preferir
            s.NUMORIGINAL
        FROM SIMILARIDADE s
        JOIN PRODUTO p ON p.CDPRODUTO = s.CDPRODUTO
        ORDER BY p.NUMORIGINAL
    """
    cursor.execute(query)
    rows = cursor.fetchall()

    # 3. Agrupar as similaridades por produto
    #    rows deve conter [(nome_produto, num_original), (...), ...]
    similaridades_por_produto = defaultdict(list)
    for nome_produto, num_original in rows:
        similaridades_por_produto[nome_produto].append(num_original)

    # 4. Criar a planilha Excel
    wb = Workbook()
    ws = wb.active

    # Ajustar larguras das colunas, se desejar
    ws.column_dimensions['A'].width = 40
    ws.column_dimensions['B'].width = 40

    current_row = 1

    for produto, lista_similares in similaridades_por_produto.items():
        if not lista_similares:
            continue

        # Mescla a coluna A para a quantidade de linhas que correspondem a este produto
        #
        # Exemplo: se há 3 similaridades para o produto X, mesclar de current_row até current_row + 2.
        #
        first_row = current_row
        last_row = current_row + len(lista_similares) - 1

        # Mesclar somente se houver mais de 1 similaridade; se houver apenas 1, não há a necessidade de mescla,
        # mas pode deixar mesclar pois ele mescla "uma linha só" sem problemas.
        ws.merge_cells(start_row=first_row, start_column=1, end_row=last_row, end_column=1)

        # Escrever o nome do produto na célula mesclada (linha inicial do grupo)
        ws.cell(row=first_row, column=1, value=produto)

        # Alinhar verticalmente no topo (ou centro)
        ws.cell(row=first_row, column=1).alignment = Alignment(vertical='center')
        ws.cell(row=first_row, column=2).alignment = Alignment(vertical='center')

        # Colocar a coluna A em negrito
        ws.cell(row=first_row, column=1).font = 

        # Escrever cada similaridade na coluna B
        for i, similaridade in enumerate(lista_similares):
            ws.cell(row=first_row + i, column=2, value=similaridade)

        # Avançar current_row
        current_row += len(lista_similares)

        current_row += 1

    # 5. Salvar e fechar conexão
    wb.save(output_path)
    conn.close()
    print(f"Planilha gerada: {output_path}")

In [14]:
gerar_planilha_similaridade("similaridades.xlsx")

Planilha gerada: similaridades.xlsx
